1\. **Pandas DataFrame**

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every `x` BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [ ]:
# If haven't downloaded it yet, please get the data file with wget
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

In [2]:
import pandas as pd
import numpy as np
import random
import datetime as dt
import matplotlib.pyplot as plt


 16750K .......... .......... .......... .......... .......... 51% 1,81M 7s
 16800K .......... .......... .......... .......... .......... 52% 1,65M 7s
 16850K .......... .......... .......... .......... .......... 52% 3,17M 6s
 16900K .......... .......... .......... .......... .......... 52% 4,24M 6s
 16950K .......... .......... .......... .......... .......... 52% 2,89M 6s
 17000K .......... .......... .......... .......... .......... 52% 3,05M 6s
 17050K .......... .......... .......... .......... .......... 52% 4,21M 6s
 17100K .......... .......... .......... .......... .......... 52% 1,52M 6s
 17150K .......... .......... .......... .......... .......... 53% 2,63M 6s
 17200K .......... .......... .......... .......... .......... 53% 2,40M 6s
 17250K .......... .......... .......... .......... .......... 53% 2,80M 6s
 17300K .......... .......... .......... .......... .......... 53% 7,32M 6s
 17350K .......... .......... .......... .......... .......... 53% 2,02M 6s
 17400K ...

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [3]:
data_file = "./data/data_000637.txt"
rows = len(open(data_file).readlines()) - 1
N = random.randint(10000, rows)
print("N =",N)
df = pd.read_csv(data_file, nrows=N)
df

N = 593002


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
592997,1,0,58,3869206171,562,3
592998,1,0,62,3869206171,559,18
592999,1,0,60,3869206171,563,5
593000,1,0,59,3869206171,609,3


2\. Estimate the number of BX in a ORBIT (the value `x`).

*Hint*: check when the BX counter reaches the maximum value before being reset to 0.

In [4]:
x = (df["BX_COUNTER"]).max()
print("Estimation of #BT in ORBIT = ",x)

Estimation of #BT in ORBIT =  3563


3\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition, and convert the new column to a Time Series.

In [5]:
df['ABS_TIME'] = (df['ORBIT_CNT'] * x * 25) + (df['BX_COUNTER']*25) + ((25 / 30) * df['TDC_MEAS'])
df['ABS_TIME_SERIES'] = pd.to_datetime(df['ABS_TIME'])
df

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,ABS_TIME,ABS_TIME_SERIES
0,1,0,123,3869200167,2374,26,3.446490e+14,1970-01-04 23:44:09.004934896
1,1,0,124,3869200167,2374,27,3.446490e+14,1970-01-04 23:44:09.004934897
2,1,0,63,3869200167,2553,28,3.446490e+14,1970-01-04 23:44:09.004939373
3,1,0,64,3869200167,2558,19,3.446490e+14,1970-01-04 23:44:09.004939490
4,1,0,64,3869200167,2760,25,3.446490e+14,1970-01-04 23:44:09.004944545
...,...,...,...,...,...,...,...,...
592997,1,0,58,3869206171,562,3,3.446495e+14,1970-01-04 23:44:09.539695877
592998,1,0,62,3869206171,559,18,3.446495e+14,1970-01-04 23:44:09.539695815
592999,1,0,60,3869206171,563,5,3.446495e+14,1970-01-04 23:44:09.539695904
593000,1,0,59,3869206171,609,3,3.446495e+14,1970-01-04 23:44:09.539697052


4\. Find out the duration of the data taking in hours, minutes and seconds, by using the features of the Time Series. Perform this check reading the whole dataset.

In [6]:
last_line_dataset = len(df) - 1
duration = df['ABS_TIME_SERIES'][last_line_dataset] - df['ABS_TIME_SERIES'][0]
print("Duration:",duration)

Duration: 0 days 00:00:00.534762192


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [7]:
groupTDC = df.groupby(['TDC_CHANNEL'])['TDC_CHANNEL'].size().sort_values(ascending=False).head(3)

print(groupTDC)



TDC_CHANNEL
139    49285
64     29660
63     29119
Name: TDC_CHANNEL, dtype: int64


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit).

In [8]:
print(len(df.groupby(['ORBIT_CNT']).count()))

6002


7\. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [9]:
print(len(df[df['TDC_CHANNEL'] == 139]['ORBIT_CNT'].unique()))

5979


8\. Create two Series (one for each FPGA) that have the TDC channel as index, and the number of counts for the corresponding TDC channel as values.

In [10]:
print(">> Series 1)\n \tFPGA = 0: \n\n ", pd.Series(df[df['FPGA']==0].groupby('TDC_CHANNEL').size()))
print("\n\n>> Series 2)\n \tFPGA = 1: \n\n ", pd.Series(df[df['FPGA']==1].groupby('TDC_CHANNEL').size()))


>> Series 1)
 	FPGA = 0: 

  TDC_CHANNEL
1        551
2        707
3        696
4       1043
5        662
       ...  
129        2
130       13
137       10
138       11
139    34557
Length: 124, dtype: int64


>> Series 2)
 	FPGA = 1: 

  TDC_CHANNEL
1      12856
2      14742
3       9946
4      11887
5       6734
       ...  
129       12
130       15
137       13
138       13
139    14728
Length: 132, dtype: int64


9\. **Optional:** Create two histograms (one for each FPGA) that show the number of counts for each TDC channel.